In [ ]:
pip install --upgrade --quiet google-cloud-aiplatform[agent_engines,adk]

In [ ]:
pip install -U -q "google-genai"

In [ ]:
pip install --quiet gradio

In [ ]:
# --- Set environment variables
import os

os.environ["GOOGLE_CLOUD_PROJECT"] = "content-creation-agent-468223"    #Name of Your Project
os.environ["GOOGLE_CLOUD_LOCATION"] = "us-central1"                     #Location
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "True"                        #Keep it to true to use Vertex AI
os.environ["GOOGLE_CLOUD_BUCKET"] = "gs://launchpad_marketing"          #Bucket used for staging
os.environ["GEMINI_VERSION"] = "gemini-2.0-flash"            #Gemini version: thie model might yield better quality results in terms of nuance, coherence, and creative flair.

In [ ]:
# --- Create Blog Post LLM Agent
from google.adk.agents import LlmAgent
from google.genai import types
from vertexai.preview.reasoning_engines import AdkApp

# --- Set safety settings:
safety_settings = [
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=types.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    ),
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_CIVIC_INTEGRITY,
        threshold=types.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    ),
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_HARASSMENT,
        threshold=types.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    ),
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
        threshold=types.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    ),
]


blog_post_generation_config = types.GenerateContentConfig(
   safety_settings = safety_settings,
   temperature = 0.75,          # Encourage creativity, stay aligned
   max_output_tokens = 800,    # Allow for 400-500 words comfortably
   top_p = 0.85,               # Balance diversity and relevance
   presence_penalty = 0.15,    # Small penalty to encourage new ideas/words
   frequency_penalty = 0.1,    # Small penalty to reduce repetition of phrases   
)


blog_post_instructions = """
You are a junior copywriter tasked with creating a compelling product announcement blog post.
Goal: Write an engaging, well-structured blog post of 400–500 words based on the provided product brief.

Your responsibilities:
1.  **Accept and Integrate Product Brief:** You will be provided with a product brief containing:
    *   Product Name: The official name of the product.
    *   Short Description: A concise summary of what the product is.
    *   Key Features: A list of the product's main functionalities or attributes
    *   Target Audience: A description of the intended readers.

2.  **Content Crafting:**
    *   Craft the content in a clear, accessible, and enthusiastic tone suited for the **Target Audience**.
    *   Elaborate on the **Key Features**, translating them into tangible benefits and solving problems for the **Target Audience**.
    *   **Infer and highlight the product's unique advantages** based on the provided description and features, without needing an explicit USP statement from the user.

3.  **Structure the Blog Post:**
    *   **Title:** Create a compelling, SEO-friendly title that clearly communicates the product launch and generates curiosity.
    *   **Introduction:** Start with a relatable problem or aspiration of the **Target Audience**, then introduce the product as the solution, briefly mentioning its core purpose. Hook the reader's interest.
    *   **Body:** Expand on the product’s benefits, use cases, and **Key Features** in 2–3 well-developed paragraphs. Each paragraph should ideally focus on a key benefit or set of related features. Emphasize what makes the product stand out based on the provided information.
    *   **Conclusion:** Summarize the main selling points and the core value proposition of the product. End with a general, encouraging closing statement that prompts further exploration or interest, without a specific "call to action" directive.

4.  **Tone and Language:**
    *   Avoid overly technical jargon unless the **Target Audience** is explicitly technical.
    *   Maintain a friendly, enthusiastic, yet professional tone throughout the post. Convey excitement about the product without making unsubstantiated claims.

Output Format:
*   Provide the blog post as markdown, using clear section headings for Title, Introduction, Body, and Conclusion.
*   Do not include any placeholder text or filler phrases; fully write out all sections with complete sentences and paragraphs.
*   Strive for a word count between 400 and 500 words, ensuring each sentence adds value and contributes to the overall narrative.
"""

blog_post_agent = LlmAgent(
    name="BlogPostAgent", 
    model = os.getenv("GEMINI_VERSION"),
    description=("Agent that writes a 400-500 word announcement blog post"),
    instruction=blog_post_instructions, 
    generate_content_config=blog_post_generation_config,
    output_key="result")

blog_app = AdkApp(agent=blog_post_agent)
print("✅ BlogPostAgent is ready to be deployed")

In [ ]:
# --- Local test of Blog Agent
for event in blog_app.stream_query(
    user_id="user_123",
    message="""I want to announce a new product called VisionLink Pro.
                It’s an AI-powered video conferencing platform that improves communication with real-time translation, automatic meeting summaries, and calendar integration.
                The main features include support for 25+ languages, Slack + Google Calendar integration, and high-definition audio and video.
                The target audience is remote teams, global companies, and project managers.
                """,):
    print(event)

In [ ]:
# --- Create Social Media LLM Agent
from google.adk.agents import LlmAgent, ParallelAgent
from google.genai import types
from vertexai.preview.reasoning_engines import AdkApp

social_media_generation_config = types.GenerateContentConfig(
    safety_settings=safety_settings, # Reuse your existing safety settings
    temperature=0.85,          # Higher temp for more creative variations
    max_output_tokens=400,     # Enough for 3 short posts + JSON structure
    top_p=0.9,                 # Allow more diversity for platform variations
    presence_penalty=0.2,      # Encourage more distinct phrasing across platforms
    frequency_penalty=0.15,    # Reduce repetition within and between posts
)

social_media_instructions = """
You are a Social Media Coordinator tasked with creating a set of promotional posts for a new product launch.
Goal: Generate a variety of engaging, platform-specific social media posts based on the provided product brief. If specific platforms are requested, generate posts for those; otherwise, default to LinkedIn, X/Twitter, and Instagram. The output must be in JSON format.

Your responsibilities:
1.  **Accept and Integrate Product Brief and Target Platforms:** You will be provided with:
    *   A product brief containing:
        *   Product Name: The official name of the product.
        *   Short Description: A concise summary of what the product is.
        *   Key Features: A list of the product's main functionalities or attributes.
        *   Target Audience: A description of the intended readers.
    *   Optionally, a list of target social media platforms provided by the user. If no platforms are provided, **default to LinkedIn, X/Twitter, and Instagram.**

2.  **Content Crafting & Platform Adaptation:**
    *   **Analyze the Product Brief:** Understand the product's core value and its appeal to the `Target Audience`.
    *   **Platform Tailoring:** For **each** platform specified by the user (or the default platforms if none are specified), generate a distinct post optimized for that platform. Consider their unique tones, lengths, and best practices.
    *   **Supported Platforms & Styles:**
        *   **LinkedIn:** Professional, benefit-driven, suitable for a business/professional audience. Focus on problem-solving and value proposition. Use relevant professional hashtags.
        *   **X/Twitter:** Concise, attention-grabbing, and punchy. Use relevant trending or product-specific hashtags. Aim for high impact in a short space.
        *   **Instagram:** Engaging, visually-oriented caption that complements an assumed product image. Use a friendly, enthusiastic tone. Include relevant hashtags for broader reach and engagement.
        *   **Facebook:** Conversational, potentially slightly longer than Twitter, but still engaging. Can include links and a good mix of benefits and enthusiasm. Use relevant hashtags.
        *   **(Add other platforms here if you want to pre-define their styles, e.g., TikTok, Pinterest)**
    *   **Infuse Product Highlights:** Weave in key benefits derived from the `Key Features` and the product's essence into each post.

3.  **Output Structure:**
    *   Your final output must be a **JSON object**.
    *   The JSON object should contain keys corresponding to each requested or defaulted platform (e.g., `linkedin_post`, `twitter_post`, `instagram_caption`, `facebook_post`). Use lowercase, snake_case for platform names in the keys.
    *   Example JSON structure:
        ```json
        {
          "linkedin_post": "Your professionally crafted LinkedIn post text here...",
          "twitter_post": "Your concise and punchy X/Twitter post text here...",
          "instagram_caption": "Your engaging Instagram caption text here..."
        }
        ```
    *   Ensure the text within each platform's key is a complete and well-formed social media post.

4.  **Tone and Language:**
    *   Maintain an enthusiastic and clear tone across all platforms.
    *   Adapt the tone slightly for each platform as described above.
    *   Avoid overly technical jargon unless the `Target Audience` specifically indicates a technical focus.
    *   Use emojis judiciously where appropriate for the platform and tone.

Output Format:
*   Provide the output as a JSON string.
*   Do not include any introductory or concluding remarks outside of the JSON object itself.
*   Ensure the generated posts are of appropriate length for each platform:
    *   LinkedIn: Aim for impactful, professional content.
    *   X/Twitter: Under 280 characters.
    *   Instagram Caption: Core message within the first few lines, engaging tone.
    *   Facebook: Conversational, engaging.
    *   **(Adjust length guidelines for any other platforms you add)**
"""

social_media_agent = LlmAgent(
    name="SocialMediaAgent", 
    model = os.getenv("GEMINI_VERSION"),
    description=("Agent that generates multi-platform short-form posts in structured JSON."),
    instruction=social_media_instructions, 
    generate_content_config=social_media_generation_config,
    output_key="result")

social_app = AdkApp(agent=social_media_agent)
print("✅ SocialMediaAgent is ready to be deployed")

In [ ]:
#--- Local Test of Social Media Agent
for event in social_app.stream_query(
    user_id="user_123",
    message="""
    I want to announce a new product called VisionLink Pro.
    It’s an AI-powered video conferencing platform that improves communication with real-time translation, automatic meeting summaries, and calendar integration.
    The main features include support for 25+ languages, Slack + Google Calendar integration, and high-definition audio and video.
    The target audience is remote teams, global companies, and project managers.
    """,):
    print(event)

In [ ]:
# --- Create Coordinator LLM Agent
campaign_coordinator_system_instructions = """
You are a Campaign Coordinator responsible for managing the end-to-end content creation workflow for a new product launch. 
Your goal is to orchestrate the generation of a comprehensive marketing package by leveraging specialized sub-agents: a Blog Post Agent and a Social Media Agent.

Goal: Your job is to understand the provided product brief and then activate the correct specialist agent(s) to generate a complete marketing package. 
You will then consolidate their outputs into a unified markdown format.

Decision Logic:

1. Receive and Process Product Brief:
    Action:** Analyze the user's input. This input will contain a product brief, which includes:
    -   Product Name
    -   Short Description
    -   Key Features
    -   Target Audience
    -   Optionally, a list of target social media platforms.

2. Identify Required Tasks and Execute Agents**
    - Action: Based on the product brief, you need to perform the following tasks:
        -   Blog Post Generation: Always generate a blog post using the **BlogPostAgent**. Use the provided product brief for this task. Ensure it follows the Blog Post Agent's specific instructions regarding length, tone, and markdown output.
        -   Social Media Content Generation: Always generate social media posts using the **SocialMediaAgent**. Use the provided product brief, and if target platforms are specified, use those. If no platforms are specified, default to LinkedIn, X/Twitter, and Instagram. Ensure it follows the Social Media Agent's instructions regarding platform tailoring, tone, and JSON output.
    - Execution Strategy:
        -   You must ensure **both** the `BlogPostAgent` and the `SocialMediaAgent` are called.
        -   You can call these agents concurrently if your framework supports it, or sequentially. The order of the calls to the sub-agents is not critical, but **both must complete** before you proceed to consolidation.
        -   Silently Activate Agents: All calls to the sub-agents (`BlogPostAgent` and `SocialMediaAgent`) must be silent. You should not inform the user that you are calling them; just execute the task.

3. Consolidate and Package Outputs:
    - Action: Once you have received the complete outputs from **both** the `BlogPostAgent` and the `SocialMediaAgent`, you must consolidate them.
    - Consolidation Method: Combine the blog post markdown and the social media JSON into a single, cohesive marketing campaign package.
    - Adherence to Agent Instructions: Ensure the content you extract and consolidate respects the output formats of the sub-agents (markdown for blog, JSON for social).

4. Tone and Context:
    - Tone: Maintain an overarching professional and organized tone as an orchestrator.
    - Context: Ensure the generated content from sub-agents is consistent with the product brief.

Core Boundaries:
    - Do Not Perform Tasks Yourself: You are an orchestrator. You must not write blog posts or social media content directly. Your job is to call the agents who do.
    - Handle Out-of-Scope Requests: If the user's request is not a product brief for marketing content generation, state that your function is specific to marketing campaign creation.
    - Agent Script for Out-of-Scope Requests: "I can help create marketing content packages based on a product brief. Please provide me with the product name, description, key features, and target audience."

Output Format:

    - The final output **must** be a single, unified markdown string.
    - Structure: Adhere strictly to the following markdown structure:

    ```markdown
    # Campaign Package: [Product Name]
    ---
    ## Blog Post
    
    (Insert the full markdown output from the Blog Post Agent here. This section should be exactly as provided by the Blog Post Agent, preserving its markdown formatting.)
    ---
    ## Social Media Posts
    
    (Here, you will present the social media posts as a markdown list. Extract the relevant text for each platform from the Social Media Agent's output and format it clearly.)
    *   **LinkedIn:** [Text of the LinkedIn post]
    *   **X/Twitter:** [Text of the X/Twitter post]
    *   **Instagram:** [Text of the Instagram caption]
    *   (Add other platforms as needed, using a similar bulleted format.)
    ---
    ```
Content Placement:
    - The `[Product Name]` placeholder should be replaced with the actual product name from the brief.
    - The blog post content should directly follow `## Blog Post\n---` and precede `---`.
    - The social media posts should follow `## Social Media Posts\n---` and be formatted as a markdown list, with each platform's post clearly labeled and presented.
    - No Extraneous Text:** Do not include any introductory or concluding remarks outside of the markdown package itself. The markdown should be the sole output.
"""

campaign_coordinator_generation_config = types.GenerateContentConfig(
    temperature=0.3,  # Lower temperature for logical orchestration, control, and reliability
    max_output_tokens=1500, # Sufficient for two agent outputs plus markdown wrapper
    top_p=0.7,        # More controlled sampling
    presence_penalty=0.0,
    frequency_penalty=0.0,
)

coordinator_agent = LlmAgent(
    name="CoordinatorAgent",
    model = os.getenv("GEMINI_VERSION"),
    instruction=campaign_coordinator_system_instructions,
    description="Campaign Coordinator",
    generate_content_config=campaign_coordinator_generation_config,
    sub_agents=[blog_post_agent,social_media_agent],
    output_key="result",
)

# Create an AdkApp that includes all your agents
# This allows the coordinator agent to find and use the other agents as tools.
coordinator_app = AdkApp(agent=coordinator_agent)

print("✅ CoordinatorAgent is ready to be deployed")

In [ ]:
#--- Local test for Coordinator Agent
for event in coordinator_app.stream_query(
    user_id="user_123",
    message=""" Product Name: VisionLink Pro.
                It’s an AI-powered video conferencing platform that improves communication with real-time translation, automatic meeting summaries, and calendar integration.
                The main features include support for 25+ languages, Slack + Google Calendar integration, and high-definition audio and video.
                The target audience is remote teams, global companies, and project managers.
                Platforms: LinkedIn, Instagram
                """,):
    print(event)

In [ ]:
# --- Deploy all Agents in Vertex AI Agent Engine (Wait for Creation of all agents before moving on...)
import vertexai

from vertexai import agent_engines

# --- Init Vertex AI Environment ---
vertexai.init(
    project= os.getenv("GOOGLE_CLOUD_PROJECT"),
    location= os.getenv("GOOGLE_CLOUD_LOCATION"),
    staging_bucket= os.getenv("GOOGLE_CLOUD_BUCKET"), 
)

# --- Deploy Blog Post Agent ---
remote_blog_agent = agent_engines.create(
    blog_app,
    display_name="Blog Post Agent",
    description="Writes 400–600 word product announcement posts in Markdown.",
    requirements=["google-cloud-aiplatform[agent_engines,adk]"],  #"cloudpickle==3.1.1","pydantic==2.11.7"
)
print("✅ Blog Post Agent created on Vertex AI Agent Engine")

# --- Deploy Social Media Agent ---
remote_social_agent = agent_engines.create(
    social_app,
    display_name="Social Media Agent",
    description="Generates multi-platform short-form posts in structured JSON from a product brief.",
    requirements=["google-cloud-aiplatform[agent_engines,adk]"], 
)
print("✅ Social Media Agent created on Vertex AI Agent Engine")

# --- Deploy Campaign Coordinator Agent ---
remote_campaign_coordinator_agent = agent_engines.create(
    coordinator_app,
    display_name="Campaign Coordinator Agent",
    description="Orchestrates Blog Post and Social Media Agents to produce a unified campaign package.",
    requirements=["google-cloud-aiplatform[agent_engines,adk]"],
)
print("✅ Campaign Coordinator Agent created on Vertex AI Agent Engine")


In [ ]:
# --- Test consolidation of Blog + Social Media Agent
remote_session = remote_blog_agent.create_session(user_id="user_123")

for event in remote_blog_agent.stream_query(
    user_id="user_123",
    session_id = remote_session["id"],
    message="""I want to announce a new product called VisionLink Pro.
                It’s an AI-powered video conferencing platform that improves communication with real-time translation, automatic meeting summaries, and calendar integration. 
                The main features include support for 25+ languages, Slack + Google Calendar integration, and high-definition audio and video. 
                The target audience is remote teams, global companies, and project managers."""
   ):
    print(event)

In [ ]:
# ============================================================
# Gradio GUI – Multi-Agent Chat (Coordinator / Blog / Social)
# - Per-agent sessions on Agent Engine
# - Chatbot (messages mode) on the right with markdown rendering
# - Left pane: agent picker, prompt, Send, Start New Session
# - Each agent returns a JSON with a "result" key (markdown or JSON)
# ============================================================

import json
import base64
import gradio as gr

USER_ID = "user_123"

# ---- Plug in your deployed remote agent handles here ----
REMOTE_HANDLES = {
    "Campaign Coordinator Agent": remote_campaign_coordinator_agent,
    "Blog Post Agent": remote_blog_agent,
    "Social Media Agent": remote_social_agent,
}

# ---------- Helpers to call engines (SDK streaming) ----------

def _extract_text_from_content(content: dict) -> str:
    parts = (content or {}).get("parts") or []
    out = []
    for p in parts:
        t = p.get("text")
        if isinstance(t, str) and t.strip():
            out.append(t.strip()); continue
        inline = p.get("inline_data", {})
        data = inline.get("data")
        if isinstance(data, str):
            try:
                out.append(base64.b64decode(data).decode("utf-8", errors="ignore").strip())
            except Exception:
                pass
    return "\n".join([x for x in out if x])

def _extract_text_from_event(ev) -> str:
    if not isinstance(ev, dict):
        return ""
    for k in ("output_text", "result"):
        v = ev.get(k)
        if isinstance(v, str) and v.strip():
            return v.strip()
    if isinstance(ev.get("content"), dict):
        s = _extract_text_from_content(ev["content"])
        if s: return s
    if isinstance(ev.get("message"), dict) and isinstance(ev["message"].get("content"), dict):
        s = _extract_text_from_content(ev["message"]["content"])
        if s: return s
    cands = ev.get("candidates")
    if isinstance(cands, list):
        texts = []
        for c in cands:
            if isinstance(c, dict) and isinstance(c.get("content"), dict):
                s = _extract_text_from_content(c["content"])
                if s: texts.append(s)
        if texts: return "\n".join(texts)
    return ""

def call_engine_once(remote_engine_handle, prompt: str, session_id: str, user_id: str = USER_ID) -> str:
    chunks = []
    for ev in remote_engine_handle.stream_query(user_id=user_id, message=prompt, session_id=session_id):
        piece = _extract_text_from_event(ev)
        if piece:
            chunks.append(piece)
    return "".join(chunks).strip()

# ---------- "result" extraction & rendering ----------

def _unfence(s: str) -> tuple[str, str]:
    """
    If `s` is code-fenced (``` or ```lang), return (inner_text, language).
    Otherwise return (s, "").
    Handles ```markdown, ```md, ```json, etc.
    """
    s = (s or "").strip()
    if not s.startswith("```"):
        return s, ""
    lines = s.splitlines()
    if len(lines) >= 2 and lines[-1].strip() == "```" and lines[0].startswith("```"):
        # First line may be ``` or ```lang
        lang = lines[0][3:].strip().lower()  # e.g., "markdown", "json"
        inner = "\n".join(lines[1:-1]).strip()
        return inner, lang
    return s, ""

def parse_result_payload(raw_text: str):
    """
    Returns (mode, value)
      mode in {"markdown", "json", "raw"}
      value is a string (markdown) or a dict/list (json)
    Rules:
      - If payload has {"result": ...}, parse that.
      - If result is fenced as ```markdown / ```md → render as markdown.
      - If result is fenced as ```json / ```javascript / ```jsonc → try JSON.
      - If fenced without lang: try JSON first, else markdown.
      - If unfenced string: try JSON if it *looks* like JSON, else markdown.
    """
    if not isinstance(raw_text, str):
        return ("raw", str(raw_text))

    text = raw_text.strip()

    # Try whole payload as JSON
    payload = None
    if text.startswith("{") or text.startswith("["):
        try:
            payload = json.loads(text)
        except Exception:
            payload = None

    # If JSON has 'result'
    if isinstance(payload, dict) and "result" in payload:
        result = payload["result"]

        # dict/list -> JSON display
        if isinstance(result, (dict, list)):
            return ("json", result)

        # string -> possibly fenced
        if isinstance(result, str):
            inner, lang = _unfence(result)

            # Language-tagged JSON
            if lang in ("json", "jsonc", "javascript"):
                try:
                    return ("json", json.loads(inner))
                except Exception:
                    # fall back to show as fenced JSON-like text
                    try:
                        obj = json.loads(inner)
                        return ("json", obj)
                    except Exception:
                        return ("markdown", inner)

            # Language-tagged markdown
            if lang in ("markdown", "md"):
                return ("markdown", inner)

            # No/other language → try JSON first
            stripped = inner.lstrip()
            if stripped.startswith("{") or stripped.startswith("["):
                try:
                    return ("json", json.loads(inner))
                except Exception:
                    pass
            return ("markdown", inner)

        # other scalar types
        return ("raw", str(result))

    # No 'result' key. Check if *entire* text is fenced.
    inner, lang = _unfence(text)
    if inner != text:
        if lang in ("json", "jsonc", "javascript"):
            try:
                obj = json.loads(inner)
                # If that JSON itself has 'result', recurse to normalize
                if isinstance(obj, dict) and "result" in obj:
                    return parse_result_payload(json.dumps(obj))
                return ("json", obj)
            except Exception:
                return ("markdown", inner)
        if lang in ("markdown", "md"):
            return ("markdown", inner)
        # Unknown/empty lang: try JSON first, else markdown
        s2 = inner.lstrip()
        if s2.startswith("{") or s2.startswith("["):
            try:
                return ("json", json.loads(inner))
            except Exception:
                pass
        return ("markdown", inner)

    # If the whole thing parsed as JSON (but no 'result'), display as JSON
    if payload is not None:
        return ("json", payload)

    # Default: treat as markdown so headings/lists render nicely
    return ("markdown", text)

def to_markdown(value) -> str:
    """
    Format dict/list values as pretty JSON fenced in code blocks.
    Leave strings as-is so markdown renders naturally.
    """
    if isinstance(value, (dict, list)):
        return "```json\n" + json.dumps(value, indent=2, ensure_ascii=False) + "\n```"
    return str(value)


# ---------- Gradio callbacks (messages mode) ----------

def ensure_session(agent_name: str, sessions: dict):
    sessions = sessions or {}
    info = sessions.get(agent_name, {})
    sid = info.get("session_id")
    if sid:
        return sid, sessions
    remote = REMOTE_HANDLES[agent_name]
    sess = remote.create_session(user_id=USER_ID)
    sid = getattr(sess, "id", None) or (sess.get("id") if isinstance(sess, dict) else str(sess))
    sessions[agent_name] = {"session_id": sid}
    return sid, sessions

def start_new_session(agent_name: str, sessions: dict, chats: dict):
    if not agent_name:
        return "Select an agent first.", sessions, chats
    remote = REMOTE_HANDLES[agent_name]
    sess = remote.create_session(user_id=USER_ID)
    sid = getattr(sess, "id", None) or (sess.get("id") if isinstance(sess, dict) else str(sess))
    sessions = sessions or {}
    sessions[agent_name] = {"session_id": sid}
    chats = chats or {}
    chats[agent_name] = []  # messages mode: list of dicts [{role, content}]
    return f"Started a new session for **{agent_name}**.", sessions, chats

def send(agent_name: str, prompt: str, sessions: dict, chats: dict):
    if not agent_name:
        return "Please select an agent.", sessions, chats, gr.update()
    if not prompt or not prompt.strip():
        return "Type something to send.", sessions, chats, gr.update()

    sid, sessions = ensure_session(agent_name, sessions)
    remote = REMOTE_HANDLES[agent_name]

    try:
        raw = call_engine_once(remote, prompt, sid, user_id=USER_ID)
    except Exception as e:
        raw = f"**SDK error**: {e}"

    mode, value = parse_result_payload(raw)
    assistant_md = to_markdown(value) if mode == "json" else value  # don't fence markdown

    chats = chats or {}
    history = chats.get(agent_name) or []
    # messages mode
    history = history + [
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": assistant_md},
    ]
    chats[agent_name] = history

    return "", sessions, chats, gr.update(value="")

def render_chat(agent_name: str, chats: dict):
    chats = chats or {}
    return chats.get(agent_name) or []

# ---------- UI ----------

with gr.Blocks(title="LaunchPad – Multi-Agent Chat") as demo:
    gr.Markdown("## 🧠 LaunchPad – Multi‑Agent Campaign Builders")

    with gr.Row():
        # Left Pane
        with gr.Column(scale=5):
            agent_dd = gr.Dropdown(
                choices=list(REMOTE_HANDLES.keys()),
                value="Campaign Coordinator Agent",
                label="Select Agent",
            )
            prompt_tb = gr.Textbox(
                label="Prompt",
                placeholder="Paste your product brief or type a request…",
                lines=8,
            )
            with gr.Row():
                send_btn = gr.Button("Send", variant="primary")
                new_sess_btn = gr.Button("Start New Session (for selected agent)")
            status_md = gr.Markdown("", elem_id="status")

        # Right Pane
        with gr.Column(scale=7):
            chat = gr.Chatbot(
                label="Conversation",
                type="messages",          # <-- use messages mode
                height=550,
                render_markdown=True,     # <-- render markdown
                sanitize_html=False       # <-- allow markdown formatting
            )

    sessions_state = gr.State({k: {} for k in REMOTE_HANDLES.keys()})
    chats_state = gr.State({k: [] for k in REMOTE_HANDLES.keys()})  # messages per agent

    send_btn.click(
        send,
        inputs=[agent_dd, prompt_tb, sessions_state, chats_state],
        outputs=[status_md, sessions_state, chats_state, prompt_tb],
    ).then(
        render_chat,
        inputs=[agent_dd, chats_state],
        outputs=[chat],
    )

    new_sess_btn.click(
        start_new_session,
        inputs=[agent_dd, sessions_state, chats_state],
        outputs=[status_md, sessions_state, chats_state],
    ).then(
        render_chat,
        inputs=[agent_dd, chats_state],
        outputs=[chat],
    )

    agent_dd.change(
        render_chat,
        inputs=[agent_dd, chats_state],
        outputs=[chat],
    )

demo.launch(share=True)


In [ ]:
#--- Cleanup Turn Off UI
demo.close()

In [ ]:
#--- Cleanup Delete Deployed Agents
remote_blog_agent.delete(force=True)
remote_social_agent.delete(force=True)
remote_campaign_coordinator_agent.delete(force=True)